# Statistics

## Installs

In [ ]:
!pip install tabulate

## Data representation

In [ ]:
from numpy import loadtxt
lines = loadtxt('train_full.txt', dtype='str', delimiter='\t')

labels = ['id', 'sentence', 'start', 'end', 'target', 'native', 'non-native', 'native_score', 'non-native_score', 'label']

data = [dict(zip(labels, line)) for line in lines]
data

In [ ]:
word_targets = [line for line in data if len(line['target'].split()) == 1]
len(word_targets)

## Make table

In [ ]:
import numpy as np

def makeTable(features, scores, frequency, file):
    features = sorted(set([feature for feature in features if feature]))
    scores = list(sorted(set(scores)))

    info = [scores]
    for feature in features:
        line = []
        for score in scores:
            if (score, feature) not in frequency.keys():
                frequency[(score, feature)] = 0
            line.append(freq[(score, feature)])
        info.append(line)

    table = tabulate(info, headers='firstrow', showindex=features)
    with open(file, 'w') as f:
        f.write(table)

In [ ]:
def getScores(target):
    scores = []

    for line in target:
        scores.append(line['label'])
    
    return scores

scores = getScores(word_targets)

In [ ]:
def getFreq(features, scores, target):
    freq = {}
    i = 0
    j = 0

    for line in target:
        score = scores[i]
        feature = features[j]

        if (score, feature) not in freq.keys():
            freq[(score, feature)] = 0

        freq[(score, feature)] = freq[(score, feature)] + 1

        i += 1
        j += 1

    return freq

## Get word length 

In [ ]:
def getLength(target):
    freq = {}
    lengths = []

    for line in target:
        lengths.append(len(line['target']))
    return lengths

lengths = getLength(word_targets)
freq = getFreq(lengths, scores, word_targets)
makeTable(lengths, scores, freq, 'lengths-score-freq.txt')

## Get word dependency

In [ ]:
import stanza
nlp_stanza = stanza.Pipeline('en')

In [ ]:
doc = nlp_stanza(word_targets[0]['sentence'])
print(*[f'word: {word.text}\tdeprel: {word.deprel}\n' for sent in doc.sentences for word in sent.words])

In [ ]:
def getDep(line):
    doc = nlp_stanza(line['sentence'])
    pos = 0
    for sent in doc.sentences:
        for word in sent.words:
            if pos >= int(line['start']):
                return word.deprel
            pos += len(word.text) + 1

dep = []
for line in word_targets[:18]:
    dep.append(getDep(line))
#     print(f"{line['target']} {getDep(line)}")

## POS tagging

In [ ]:
def getWordID(start, sentence):
    pos = 0
    for i in range(len(sentence)):
        if i >= start:
            return pos
        if sentence[i] == ' ':
            pos += 1
    return pos

In [ ]:
from nltk import pos_tag, word_tokenize

def getPOS(line):
    pos = getWordID(int(line['start']), line['sentence'])
    text = pos_tag(word_tokenize(line['sentence']))
    for i in range(len(text)):
        if i >= pos and text[i][0] == line['target']:
            return text[i][1]

In [ ]:
pos = []
for line in word_targets:
    pos.append(getPOS(line))

In [ ]:
set(pos)

In [ ]:
freq = getFreq(pos, scores, word_targets)
makeTable(pos, scores, freq, 'pos-score-freq.txt')

In [ ]:
print(w2v_model.wv.most_similar(positive='chinese'))
print(w2v_model.wv.most_similar(positive='family'))
print(w2v_model.wv.most_similar(positive='country'))
print(w2v_model.wv.most_similar(positive='attack'))

# Basic libraries

In [ ]:
one_word_lines = [line[:5] for line in lines if len(line[4].split()) == 1]
one_word_lines

# Solve problem

In [5]:
import nltk
from nltk.corpus import brown
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\mirun\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [1]:
from numpy import loadtxt
lines = loadtxt('train_full.txt', dtype='str', delimiter='\t')
linesTest = loadtxt('test.txt', dtype='str', delimiter='\t')
# linesTest = loadtxt('News_Dev.tsv', dtype='str', delimiter='\t')

# Dataframe

In [2]:
def getTrainData():
    return [line[4].lower() for line in lines]


def getTrainLabel():
    return [line[9] for line in lines]


def getTestData():
    return [line[4].lower() for line in linesTest]


def getTestLabel():
    return [line[10] for line in linesTest]


def getDataFrame():
    train_df = pd.DataFrame(getTrainData(), columns = ['data'])
    train_df['label'] = getTrainLabel()

    test_df = pd.DataFrame(getTestData(), columns = ['data'])
#     test_df['label'] = getTestLabel()
    
    return train_df, test_df


def splitTrainTest():
    data_df = pd.DataFrame(getTrainData(), columns = ['data'])
    data_df['label'] = getTrainLabel()

    return train_test_split(data_df, test_size=0.2, shuffle = True)

# Vectorizers

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

from nltk.tokenize import word_tokenize, RegexpTokenizer
import pandas as pd
import numpy as np

from nltk.corpus import wordnet


class Vectorizer():
    vector_size = 300

    def __init__(self, model, train_df, test_df):
        self.model = model

        if model == 'tfidf':
            self.makeDatasetFromWords()

            self.vectorizer = TfidfVectorizer(lowercase = False)
            self.vectorizer.fit(self.dataset)

        elif model == 'word2vec':
            self.makeDatasetFromSentences()

            self.vectorizer = Word2Vec(sentences=self.dataset, vector_size=300, window=5, min_count=1, workers=16)
            self.vectorizer.save("word2vec.model")

        elif model == 'fastText':
            self.makeDatasetFromSentences()
            
            self.vectorizer = FastText(vector_size=300, window=5, min_count=1)
            self.vectorizer.build_vocab(corpus_iterable=self.dataset)
            self.vectorizer.train(corpus_iterable=self.dataset, total_examples=len(self.dataset), epochs=10)

        elif model == 'semantic':
            self.makeDatasetFromWords()


        self.vectorize(train_df, test_df)


#     def makeDatasetFromSentences(self):
#         self.dataset = np.array(list(set([line[1].lower() for line in lines] + [line[1].lower() for line in linesTest])))
#         self.preprocessSentences()

    def makeDatasetFromSentences(self):
        brownData = [' '.join(sentence).lower() for sentence in brown.sents()]
        self.dataset = np.array(list(set([line[1].lower() for line in lines] + [line[1].lower() for line in linesTest] + brownData)))
        self.preprocessSentences()


    def makeDatasetFromWords(self):
        self.dataset = np.array(list(set(
            [line[1].lower() for line in lines] + [line[1].lower() for line in linesTest] + \
            [line[4].lower() for line in lines] + [line[4].lower() for line in linesTest] \
        )))
        self.preprocessWords()


    def preprocessWords(self):
        self.dataset = np.array(list(set(np.hstack([self.tokenizePhrase(phrase) for phrase in self.dataset]))))


    def preprocessSentences(self):
        tokens = set([' '.join(self.tokenizePhrase(phrase)) for phrase in self.dataset])
        self.dataset = np.array([tokensList.split() for tokensList in tokens])

#     def preprocess(self):
#         tokenizer = RegexpTokenizer(r'[a-zA-Z\'-]+')
#         self.dataset = np.array(list(set(np.hstack([self.tokenizePhrase(phrase) for phrase in self.dataset]))))


    def tokenizePhrase(self, phrase):
        tokenizer = RegexpTokenizer(r'[a-zA-Z\'-]+')
        return word_tokenize(' '.join(tokenizer.tokenize(phrase)))


    def vectorize(self, train_df, test_df):
        if self.model == 'tfidf':
            self.X_train = self.vectorizer.transform(train_df['data']).toarray()
            self.X_test = self.vectorizer.transform(test_df['data']).toarray()


        elif self.model in ['word2vec', 'fastText']:
            self.X_train = np.array([self.vectorizePhrase(phrase, self.vectorizer.wv.get_vector) for phrase in train_df['data']])
            self.X_test = np.array([self.vectorizePhrase(phrase, self.vectorizer.wv.get_vector) for phrase in test_df['data']])
            
        elif self.model == 'semantic':
            self.X_train = np.array([self.vectorizePhrase(phrase, self.extractFeatures) for phrase in train_df['data']])
            self.X_test = np.array([self.vectorizePhrase(phrase, self.extractFeatures) for phrase in test_df['data']])
            
        self.y_train = np.array(train_df['label'], dtype='float')
#         self.y_test = np.array(test_df['label'], dtype='float')


    def vectorizePhrase(self, phrase, func):
        tokenizer = RegexpTokenizer(r'[a-zA-Z\'-]+')
        vectorizations = np.array([func(word) for word in self.tokenizePhrase(phrase)])

        return np.mean(vectorizations, axis=0)


    def extractFeatures(self, word):
        vowels = sum(map(word.count, 'aeiou'))
        length = len(word)
        doubleLetters = sum([word[i - 1] == word[i] for i in range(1, length)])
        
        maxConsecutiveConsonants = 0
        localSum = 0
        for l in word:
            if l not in 'aeiou':
                maxConsecutiveConsonants = max(maxConsecutiveConsonants, localSum)
                localSum = 0
            else:
                localSum += 1
        maxConsecutiveConsonants = max(maxConsecutiveConsonants, localSum)
        
#         senses = len(wordnet.synsets(word))

        return [length, vowels / length, doubleLetters, maxConsecutiveConsonants]

In [6]:
train_df, test_df = getDataFrame()
# tfidf = Vectorizer('tfidf', train_df, test_df)
w2v = Vectorizer('word2vec', train_df, test_df)

In [7]:
features = Vectorizer('semantic', train_df, test_df)

# Test class

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVR, LinearSVR, NuSVR
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

from matplotlib.pyplot import hist


class Solve:
    def __init__(self, vectorizer):
        getDataFrame()
#         self.splitTrainTest()

        self.X_train = vectorizer.X_train
        self.X_test = vectorizer.X_test

        self.y_train = vectorizer.y_train
#         self.y_test = vectorizer.y_test

        self.X_train = np.c_[self.X_train, features.X_train]
        self.X_test = np.c_[self.X_test, features.X_test]

        self.y_train = self.y_train.reshape(-1, 1)
#         self.y_test = self.y_test.reshape(-1, 1)

#         self.getScores()
        self.findScore()


    def scaleData(self, y_scaler):
        X_scaler = StandardScaler()
        X_train = X_scaler.fit_transform(self.X_train)
        X_test = X_scaler.transform(self.X_test)

        y_train = y_scaler.fit_transform(self.y_train)
#         y_test = y_scaler.transform(self.y_test)

        return X_train, X_test, y_train


    # - Train --------------------------------------------------

    def train(self, model, to_scale):
        if to_scale:
            y_scaler = StandardScaler()
            X_train, X_test, y_train, y_test = self.scaleData(y_scaler)

            model.fit(X_train, y_train)
            return self.scaled_mae(y_train, model.predict(X_train), y_scaler), \
                self.scaled_mae(y_test, model.predict(X_test), y_scaler)

        else:
            model.fit(self.X_train, self.y_train)
            return mean_absolute_error(self.y_train, model.predict(self.X_train)), \
                mean_absolute_error(self.y_test, model.predict(self.X_test))


    def scaled_mae(self, y_true, y_pred, scaler):
        return mean_absolute_error(scaler.inverse_transform(y_true), scaler.inverse_transform(y_pred))

    
    def printScore(self, model, model_name, to_scale = True):
        train_score, test_score = self.train(model, to_scale)

        print(f'{model_name}: \n' + \
             f'Train MAE: {train_score} \n' + \
             f'Test MAE: {test_score} \n')


    def getScores(self):
#         self.printScore(LogisticRegression(), "Logistic Regression", False)
#         self.printScore(NuSVR(C=10, coef0=1.0), "Nu-SVR")
#         self.printScore(NuSVR(C=10, coef0=1.0), "Nu-SVR - no scaling", False)
#         self.printScore(LinearSVR(C=5), "Linear SVR")
#         self.printScore(LinearSVR(C=5), "Linear SVR - no scaling", False)
#         self.printScore(Ridge(alpha=1.0), "Ridge")
#         self.printScore(Ridge(alpha=1.0), "Ridge - no scaling", False)
        self.printScore(KNeighborsRegressor(n_neighbors=2), f"kNN + features")
        self.printScore(KNeighborsRegressor(n_neighbors=2), f"kNN + features - no scaling", False)
#         self.printScore(RandomForestRegressor(n_estimators=1000, max_depth=300, random_state=0), "Forest - no scaling", False)
#         self.printScore(RandomForestRegressor(n_estimators=1000, max_depth=300, random_state=0), "Forest")


    def findScore(self):
        y_scaler = StandardScaler()
        X_train, X_test, y_train = self.scaleData(y_scaler)
        
        model = KNeighborsRegressor(n_neighbors=2)
        model.fit(X_train, y_train)

        self.res = y_scaler.inverse_transform(model.predict(X_test))
        np.savetxt('./submissions/submission4.csv', np.c_[[line[0] for line in linesTest], self.res], delimiter=',', header='id,label', comments='', fmt='%s')

In [9]:
solutions = Solve(w2v)

In [37]:
solutions = Solve(tfidf)

kNN: 
Train MAE: 0.04981788315954863 
Test MAE: 0.07103174603174603 

kNN - no scaling: 
Train MAE: 0.049177498452602005 
Test MAE: 0.07015306122448979 



In [39]:
solutions = Solve(tfidf)

kNN + features: 
Train MAE: 0.05073203828024568 
Test MAE: 0.07240173847316705 

kNN + features - no scaling: 
Train MAE: 0.04570180450411847 
Test MAE: 0.06779100529100529 



In [5]:
solutions = Solve('word2vec')

KeyError: "Key 'barren' not present"

In [27]:
solutions = Solve('semantic')

C:\Users\mirun\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mirun\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\mirun\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Linear SVR: 
Train MAE: 0.08793877766907492 
Test MAE: 0.08937679791441001 



C:\Users\mirun\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Linear SVR - no scaling: 
Train MAE: 0.14775185495883286 
Test MAE: 0.14767033132083182 

Ridge: 
Train MAE: 0.10599724255115797 
Test MAE: 0.10616548577217227 

Ridge - no scaling: 
Train MAE: 0.10599765861633148 
Test MAE: 0.10616541260915204 

kNN: 
Train MAE: 0.0815719182973861 
Test MAE: 0.09128684807256236 

kNN - no scaling: 
Train MAE: 0.07958005999142981 
Test MAE: 0.08863945578231293 



In [5]:
solutions = Solve('tfidf')

C:\Users\mirun\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibSVM]Nu-SVR: 
Train MAE: 0.040018643439954746 
Test MAE: 0.06639279337937004 



C:\Users\mirun\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibSVM]Nu-SVR - no scaling: 
Train MAE: 0.021147066881803674 
Test MAE: 0.06102471514487684 

